In [1]:
from vipy.dataset.meva import Mevadata_Public_01
from vipy.batch import Batch
from vipy.util import flatlist, groupbyasdict
from vipy.visualize import montage

In [ ]:
d = Mevadata_Public_01(videodir='/Volumes/JEBYRNE/meva/Mevadata_Public_01',  # http://mevadata.org/#getting-data
                       repodir='/Volumes/JEBYRNE/meva/meva-data-repo')       # http://gitlab.kitware.com/meva/meva-data-repo
meva = d.MEVA(verbose=False)

In [ ]:
activitylist = flatlist([m.activityclip(padframes=(0,100)) for m in meva])          # Videos to activity clips
d_category_to_activitylist = groupbyasdict(activitylist, lambda im: im.category())  # Group clips by activity label
shortlist = [a[0] for (k,a) in d_category_to_activitylist.items()]                  # One activity per category

In [ ]:
imb = Batch(shortlist, n_processes=2)                                               # local parallelism
framelist = flatlist(imb.map(lambda a: [a.clone(flushbackward=True)[k]              # Interpolate kth frame
                                         .centersquare()                            # Crop the center square per frame                                        
                                         .mindim(256)                               # Set square dimension to 256  
                                         .savefig(fontsize=10, boxalpha=0.5).rgb()  # Overlay annotations  
                                        for k in range(0,100,10)]))                 # Show every 10 frames of activity

In [ ]:
# Display one example of each activity by row, every ten frames by column for one hundred frames
montage(framelist, imgwidth=256, imgheight=256, gridrows=len(d_category_to_activitylist), gridcols=10).show()
print([a.category() for a in imb.batch()])